In [1]:
import math

In [2]:
from sympy import *
from sympy.vector import CoordSys3D,  matrix_to_vector
from sympy.matrices import Transpose
from sympy.solvers.ode.systems import dsolve_system
init_printing(use_unicode=True)

In [3]:
def get_T_a(dummy):
    roll = dummy[0]
    pitch = dummy[1]
    yaw = dummy[2]
    return Matrix([
        [cos(pitch)*cos(yaw),sin(roll)*sin(pitch)*cos(yaw)-cos(roll)*sin(yaw),cos(roll)*sin(pitch)*cos(yaw)+sin(roll)*sin(yaw)],
        [cos(pitch)*sin(yaw),sin(roll)*sin(pitch)*sin(yaw)+cos(roll)*cos(yaw),cos(roll)*sin(pitch)*sin(yaw)-sin(roll)*cos(yaw)],
        [-sin(pitch),sin(roll)*cos(pitch),cos(roll)*cos(pitch)]
    ])
def get_T_r(dummy):
    roll = dummy[0]
    pitch = dummy[1]
    yaw = dummy[2]
    return Matrix([
        [1, sin(roll)*tan(pitch),cos(roll)*tan(pitch)],
        [0, cos(roll),-sin(roll)],
        [0,sin(roll)*sec(pitch),cos(roll)*sec(pitch)]
    ])
def get_T_a_12(dummy):
    return get_T_a(dummy)
def get_T_a_23(beta_2):
    return get_T_a(Matrix([[0],[0],[-beta_2]]))
def get_T_a_34(alpha_3):
    return get_T_a(Matrix([[0],[-alpha_3],[0]]))
def get_T_r_12(dummy):
    return get_T_a(dummy)
def get_T_r_23(beta_2):
    return get_T_a(Matrix([[0],[0],[-beta_2]]))
def get_T_r_34(alpha_3):
    return get_T_a(Matrix([[0],[-alpha_3],[0]]))
def get_mag_of_vector(matrix):
    C = CoordSys3D('C')
    v = matrix_to_vector(matrix, C)
    return v.magnitude()
def get_C_drag(alpha_3):
    return 1 #TODO
def get_C_lift(alpha_3):
    return 1 #TODO
def get_C_Yr(r,mag):
    return 1 #TODO
def get_C_M(alpha_3):
    return 1 #TODO
def get_derivative_of_matrix(matrix):
    return Matrix([
        [Derivative(matrix[0],t)],
        [Derivative(matrix[1],t)],
        [Derivative(matrix[2],t)]
    ])
def multiply_sqare_matrix(test1,test2):
    return Matrix([
        [
            test1[0,0]*test2[0,0]+test1[1,0]*test2[0,1]+test1[2,0]*test2[0,2],
            test1[0,1]*test2[0,0]+test1[1,1]*test2[0,1]+test1[2,1]*test2[0,2],
            test1[0,2]*test2[0,0]+test1[1,2]*test2[0,1]+test1[2,2]*test2[0,2]
        ],
        [
            test1[0,0]*test2[1,0]+test1[1,0]*test2[1,1]+test1[2,0]*test2[1,2],
            test1[0,1]*test2[1,0]+test1[1,1]*test2[1,1]+test1[2,1]*test2[1,2],
            test1[0,2]*test2[1,0]+test1[1,2]*test2[1,1]+test1[2,2]*test2[1,2]
        ],
        [
            test1[0,0]*test2[2,0]+test1[1,0]*test2[2,1]+test1[2,0]*test2[2,2],
            test1[0,1]*test2[2,0]+test1[1,1]*test2[2,1]+test1[2,1]*test2[2,2],
            test1[0,2]*test2[2,0]+test1[1,2]*test2[2,1]+test1[2,2]*test2[2,2]
        ]
    ])
def multiply_matrix_and_vecor(test1,test2):
    return Matrix([
        [
            test1[0,0]*test2[0]+test1[0,1]*test2[1]+test1[0,2]*test2[2]
        ],
        [
            test1[1,0]*test2[0]+test1[1,1]*test2[1]+test1[1,2]*test2[2]
        ],
        [
            test1[2,0]*test2[0]+test1[2,1]*test2[1]+test1[2,2]*test2[2]
        ]
    ])


In [10]:
m = 0.1 #kg mass of the disk
d = 0.137*2 #m Diameter of frizbee
#r = 5 # rad/sec how fast its spinning
S = pi*(d**2)/4 # the area of a circle/disk_wing_reference_area
g = Matrix([[0],[0],[9.81]])
c = d
k_zz = 0.86*c/2# radius of giration
I_z = m*k_zz**2
I_x = 0.5*I_z
I_y = 0.5*I_z
I = Matrix([
    [I_x,0,0],
    [0,I_y,0],
    [0,0,I_z]
])

#TODO figure out these
C_L_p = 1
C_M_q = 1
C_N_r = 1
rho_inf = 1

t = symbols('t')
#X = MatrixSymbol('X', 3, 1)
#Y = MatrixSymbol('Y', 3, 1)

X = Function('X')(t)
Y = Function('Y')(t)
Z = Function('Z')(t)
R = Function('R')(t)
P = Function('P')(t)
W = Function('W')(t)

Position = Matrix([
    X,
    Y,
    Z
])

Angle = Matrix([
    R,
    P,
    W
])


xdot_1 = get_derivative_of_matrix(Position)
ydot_1 = get_derivative_of_matrix(Angle)

xdot_2 = get_T_a_12(Angle)*xdot_1
ydot_2 = get_T_r_12(Angle)*ydot_1

p_2 = ydot_2[0]
q_2 = ydot_2[1]
r_2 = ydot_2[2]

beta_2 = atan(xdot_2[1]/xdot_2[0])
xdot_3 = get_T_a_23(beta_2)*xdot_2
ydot_3 = get_T_r_23(beta_2)*ydot_2

alpha_3 = atan(xdot_3[2]/xdot_3[0])
xdot_4 = get_T_a_34(alpha_3)*xdot_3
ydot_4 = get_T_r_34(alpha_3)*ydot_3

p_4 = ydot_4[0]
q_4 = ydot_4[1]
r_4 = ydot_4[2]
r_3 = ydot_3[2]

C_drag = get_C_drag(alpha_3)
C_lift = get_C_lift(alpha_3)
C_slide = get_C_Yr(r_3,get_mag_of_vector(xdot_4))
C_L = C_L_p*(p_4*c/(2*get_mag_of_vector(xdot_4)))
C_M = get_C_M(alpha_3)+C_M_q*(q_4*c/(2*get_mag_of_vector(xdot_4)))
C_N = C_N_r*(r_4*c/(2*get_mag_of_vector(xdot_4)))

C_F = Matrix([-C_drag, C_slide, -C_lift])
C_mom = Matrix([C_L,C_M,C_N])
q_inf = 0.5*rho_inf*get_mag_of_vector(xdot_1)**2

M_4 = q_inf*S*c*C_mom
F_4 = q_inf*S*C_F

F_2_aero = multiply_sqare_matrix(Transpose(get_T_a_23(beta_2)),Transpose(get_T_a_34(alpha_3)))*F_4
M_2 = multiply_sqare_matrix(Transpose(get_T_r_23(beta_2)),Transpose(get_T_r_34(alpha_3)))*M_4

F_2 = F_2_aero + get_T_a_12(Angle)*m*g

w_sqwig = Matrix([
    [0,-r_2,q_2],
    [r_2,0,-p_2],
    [-q_2,p_2,0]
])

xdotdot_2 = F_2/m-w_sqwig*xdot_2
ydotdot_2 = Inverse(I)*M_2-w_sqwig*ydot_2


xdotdot_1 = multiply_matrix_and_vecor(Transpose(get_T_a_12(Angle)),xdotdot_2)
ydotdot_1 = multiply_matrix_and_vecor(Transpose(get_T_r_12(Angle)),ydotdot_2)



In [11]:

dsolve_system(
    [
        Eq(get_derivative_of_matrix(get_derivative_of_matrix(Angle))[0], ydotdot_1[0]),
        Eq(get_derivative_of_matrix(get_derivative_of_matrix(Angle))[1], ydotdot_1[1]),
        Eq(get_derivative_of_matrix(get_derivative_of_matrix(Angle))[2], ydotdot_1[2])
    ],
    [R,P,W],
    t,
    doit=False,
    simplify=False
)

KeyboardInterrupt: 

In [ ]:

dsolve_system(
    [
        Eq(get_derivative_of_matrix(get_derivative_of_matrix(Position))[0], xdotdot_1[0]),
        Eq(get_derivative_of_matrix(get_derivative_of_matrix(Position))[1], xdotdot_1[1]),
        Eq(get_derivative_of_matrix(get_derivative_of_matrix(Position))[2], xdotdot_1[2]),
        Eq(get_derivative_of_matrix(get_derivative_of_matrix(Angle))[0], ydotdot_1[0]),
        Eq(get_derivative_of_matrix(get_derivative_of_matrix(Angle))[1], ydotdot_1[1]),
        Eq(get_derivative_of_matrix(get_derivative_of_matrix(Angle))[2], ydotdot_1[2])
    ],
    [X,Y,Z,R,P,W],
    t,
    doit=False,
    simplify=False
)

# other possible solutions
# http://bjodah.github.io/pyodesys/latest/index.html
# https://www.sympy.org/scipy-2017-codegen-tutorial/notebooks/20-ordinary-differential-equations.html
# https://github.com/neurophysik/jitcode

In [ ]:

X = Matrix([
    Function('X')(t),
    Function('Y')(t),
    Function('Z')(t)
])
xdot_1 = get_derivative_of_matrix(X)
print(xdot_1)
expr_x = Eq(get_derivative_of_matrix(get_derivative_of_matrix(X))[0], xdot_1[0])

#print(expr_x)
dsolve(expr_x)

In [ ]:

f, g = symbols("f g", cls=Function)
x = symbols("x")
eqs = [Eq(f(x).diff(x), g(x)), Eq(g(x).diff(x), f(x))]
dsolve_system(eqs,[f(x),g(x)],x)

In [ ]:


d = 0.137*2 #m Diameter of frizbee
t_h = 0.038 #m Height of fizbee
pi = 3.14
S = pi*(d**2)/4 # the area of a circle/disk_wing_reference_area
AR = (d**2)/S # for disks this is about 1.27
thickness_of_chord_ratio = t_h/d # ~0.15
print(thickness_of_chord_ratio)
c_bar = pi*diameter/4 # mean_chord
mac = 8*diameter/(3*pi) # mean_aerodynamic_chord
print(mac)


In [ ]:
C_drag = drag/(q_inf*S)
C_slide = slide/(q_inf*S)
C_lift = lift/(q_inf*S)

C_L = L/(q_inf*S*c) # Rolling
C_M = M/(q_inf*S*c) # Pitching
C_N = N/(q_inf*S*c) # Yawing

X = lift*math.sin(alpha) - drag*math.cos(alpha)
Y = slide
Z = -left*math.cos(alpha) - drag*math.sin(alpha)
C_X = X/(q_inf*S*X) # axial
C_Y = X/(q_inf*S*Y) # side
C_Z = X/(q_inf*S*Z) # normal


In [ ]:
p_w = # pressure of the disk-wing surface
p_inf = # total free-stream pressure 
V_inf = # free-stream flow speed
q_inf = .5* p_inf*V_inf**2
C_p = (p_w-p_inf)/q_inf

In [ ]:
a_o = 2*pi # curve slope for an infinite wing 
e = 1 # span effectiveness factor
a = a_o / (1+(a_o/(pi*e*AR))) # lift curve slope of the finite wing
print(a) # about 2.44
alpha_o = -0.085/a# taken from experimental data at -2deg see page 33 TODO: look into this assumption
C_L = a*(alpha-alpha_o) # Due to the positive camber of a disc-wing the theoretical lift curve
S_f = t_h * d # The frontal area of the disc can be approximated from the disc centre-line thickness to chord ratio
C_Df = 0.5 # The drag coefficient of a sphere in turbulent flow
C_Do = (S_f/S)*C_Df# profile drag
print(C_Do) # ~0.0955
C_D = C_Do + C_L**2/(pi*e*AR) # this is the thoretical calc. Not the simulated calc.



In [ ]:
#L = I_x*pdot+0.5*q*r*I_z
#M = I_y*qdot-0.5*r*p*I_z
q = 2*L/(r*I_z) # angular velocity of pitching
p = -2*M/(r*I_z)  # roll rate

In [ ]:
# Non dimensional numbers
t_hat = t*g/V_inf
p_hat = p*V_inf/g
M = -k_n*C_lift*q_inf*S*d
m = 1 #kg mass of the disk
k_zz = 0.86*d/2# radius of giration
I_z = m*k_zz**2

# if lift = weight
C_lift = m*g/(q_inf*S)
AdvR = r*d/(2*V_inf)
# calc p_hat